In [358]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [359]:
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')
y_train=df_train['Survived']

In [360]:
df_train['is_test']=0
df_test['is_test']=1

In [361]:
del df_train['Survived']

In [362]:
df1=pd.concat([df_train,df_test])

In [363]:
df1.Embarked.fillna('S', inplace=True)

In [364]:
df1['has_Cabin']=df1.Cabin.notnull().astype(int)

In [365]:
df1['Cabin']=df1.Cabin.str.split()

In [366]:
df1['N_Cabin_1']=(df1.Cabin.str.len()==1).astype(int)

In [367]:
df1['N_Cabin_2']=(df1.Cabin.str.len()==2).astype(int)

In [368]:
df1['N_Cabin_3']=(df1.Cabin.str.len()==3).astype(int)

In [369]:
df1['N_Cabin_4']=(df1.Cabin.str.len()==4).astype(int)

In [370]:
del df1['Cabin']

In [371]:
df1['Name_20']=((df1.Name.str.len()<=20)).astype(int)
df1['Name_2030']=((df1.Name.str.len()>20) & (df1.Name.str.len()<=30)).astype(int)
df1['Name_3040']=((df1.Name.str.len()>30) & (df1.Name.str.len()<=40)).astype(int)
df1['Name_40']=((df1.Name.str.len()>40)).astype(int)

In [372]:
df1['is_Miss']=df1.Name.str.contains('Miss').astype(int)
df1['is_Master']=df1.Name.str.contains('Master').astype(int)
df1['is_Mr']=df1.Name.str.contains(r'(?:\s|^)Mr.(?:\s|$)').astype(int)
df1['is_Mrs']=df1.Name.str.contains(r'(?:\s|^)Mrs.(?:\s|$)').astype(int)

In [373]:
df1['Rare']=((df1.is_Miss!=1)&(df1.is_Master!=1)&(df1.is_Mr!=1)&(df1.is_Mrs!=1)).astype(int)

In [386]:
df1.Age[df1.is_Master==1].fillna((df1.Age[df1.is_Master==1]).median(), inplace=True)
df1.Age[df1.is_Miss==1].fillna((df1.Age[df1.is_Miss==1]).median(), inplace=True)
df1.Age[df1.is_Mr==1].fillna((df1.Age[df1.is_Mr==1]).median(), inplace=True)
df1.Age[df1.is_Mrs==1].fillna((df1.Age[df1.is_Mrs==1]).median(), inplace=True)

In [389]:
df1.Age[df1.is_Master==1].fillna((df1.Age[df1.is_Master==1]).median())

In [390]:
df1.Age[df1.is_Master==1]

7       2.00
16      2.00
50      7.00
59     11.00
63      4.00
65       NaN
78      0.83
125    12.00
159      NaN
164     1.00
165     9.00
171     4.00
176      NaN
182     9.00
183     1.00
193     3.00
261     3.00
278     7.00
305     0.92
340     2.00
348     3.00
386     1.00
407     3.00
445     4.00
480     9.00
489     9.00
549     8.00
709      NaN
751     6.00
755     0.67
       ...  
788     1.00
802    11.00
803     0.42
819    10.00
824     2.00
827     1.00
831     0.83
850     4.00
869     4.00
21      9.00
55     10.00
64     13.00
80      6.00
89      2.00
154    13.00
161     7.00
192    11.50
194     8.00
196     6.00
201     0.33
244      NaN
281     0.75
307     0.83
339      NaN
344      NaN
360    14.50
379     5.00
389     6.00
392    13.00
417      NaN
Name: Age, Length: 61, dtype: float64

In [391]:
df1.iterrows()

<generator object DataFrame.iterrows at 0x000000000CE6ADB0>

In [22]:
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, GridSearchCV

In [9]:
ones_ratio=y_train[y_train==1].shape[0]*1.0/y_train[y_train==0].shape[0]

param_grid = {
    # параметры ансамбля
    'n_estimators': [10, 30, 50, 100, 200, 400, 600, 1000],
    'learning_rate': [0.1],
    
    # параметры дерева
    'max_depth': [5],
    'min_child_weight': [2],
    'gamma': [0.1],
    'subsample': [0.8],
    'colsample_bytree': [0.8],
    'scale_pos_weight': [ones_ratio],
    
    # параметры регуляризации
    'reg_alpha': [0.0],
    'reg_lambda': [1.0]
}

cv = KFold(n_splits=4, shuffle=True)

clf = XGBClassifier()
gs = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=cv, verbose=5)

gs.fit(X_train, y_train)
best_params = gs.best_estimator_.get_params()
print('Best score (ACC): ', gs.best_score_)
print('Best params: ')
best_params

Fitting 4 folds for each of 8 candidates, totalling 32 fits
[CV] colsample_bytree=0.8, gamma=0.1, learning_rate=0.1, max_depth=5, min_child_weight=2, n_estimators=10, reg_alpha=0.0, reg_lambda=1.0, scale_pos_weight=0.6229508196721312, subsample=0.8 
[CV]  colsample_bytree=0.8, gamma=0.1, learning_rate=0.1, max_depth=5, min_child_weight=2, n_estimators=10, reg_alpha=0.0, reg_lambda=1.0, scale_pos_weight=0.6229508196721312, subsample=0.8, score=0.853744, total=   0.0s
[CV] colsample_bytree=0.8, gamma=0.1, learning_rate=0.1, max_depth=5, min_child_weight=2, n_estimators=10, reg_alpha=0.0, reg_lambda=1.0, scale_pos_weight=0.6229508196721312, subsample=0.8 
[CV]  colsample_bytree=0.8, gamma=0.1, learning_rate=0.1, max_depth=5, min_child_weight=2, n_estimators=10, reg_alpha=0.0, reg_lambda=1.0, scale_pos_weight=0.6229508196721312, subsample=0.8, score=0.857283, total=   0.0s
[CV] colsample_bytree=0.8, gamma=0.1, learning_rate=0.1, max_depth=5, min_child_weight=2, n_estimators=10, reg_alpha=0

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


[CV]  colsample_bytree=0.8, gamma=0.1, learning_rate=0.1, max_depth=5, min_child_weight=2, n_estimators=30, reg_alpha=0.0, reg_lambda=1.0, scale_pos_weight=0.6229508196721312, subsample=0.8, score=0.870396, total=   0.2s
[CV] colsample_bytree=0.8, gamma=0.1, learning_rate=0.1, max_depth=5, min_child_weight=2, n_estimators=30, reg_alpha=0.0, reg_lambda=1.0, scale_pos_weight=0.6229508196721312, subsample=0.8 
[CV]  colsample_bytree=0.8, gamma=0.1, learning_rate=0.1, max_depth=5, min_child_weight=2, n_estimators=30, reg_alpha=0.0, reg_lambda=1.0, scale_pos_weight=0.6229508196721312, subsample=0.8, score=0.861508, total=   0.0s
[CV] colsample_bytree=0.8, gamma=0.1, learning_rate=0.1, max_depth=5, min_child_weight=2, n_estimators=30, reg_alpha=0.0, reg_lambda=1.0, scale_pos_weight=0.6229508196721312, subsample=0.8 
[CV]  colsample_bytree=0.8, gamma=0.1, learning_rate=0.1, max_depth=5, min_child_weight=2, n_estimators=30, reg_alpha=0.0, reg_lambda=1.0, scale_pos_weight=0.6229508196721312, su

[CV]  colsample_bytree=0.8, gamma=0.1, learning_rate=0.1, max_depth=5, min_child_weight=2, n_estimators=600, reg_alpha=0.0, reg_lambda=1.0, scale_pos_weight=0.6229508196721312, subsample=0.8, score=0.853552, total=   0.5s
[CV] colsample_bytree=0.8, gamma=0.1, learning_rate=0.1, max_depth=5, min_child_weight=2, n_estimators=600, reg_alpha=0.0, reg_lambda=1.0, scale_pos_weight=0.6229508196721312, subsample=0.8 
[CV]  colsample_bytree=0.8, gamma=0.1, learning_rate=0.1, max_depth=5, min_child_weight=2, n_estimators=600, reg_alpha=0.0, reg_lambda=1.0, scale_pos_weight=0.6229508196721312, subsample=0.8, score=0.875685, total=   0.3s
[CV] colsample_bytree=0.8, gamma=0.1, learning_rate=0.1, max_depth=5, min_child_weight=2, n_estimators=600, reg_alpha=0.0, reg_lambda=1.0, scale_pos_weight=0.6229508196721312, subsample=0.8 
[CV]  colsample_bytree=0.8, gamma=0.1, learning_rate=0.1, max_depth=5, min_child_weight=2, n_estimators=600, reg_alpha=0.0, reg_lambda=1.0, scale_pos_weight=0.622950819672131

[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    3.6s finished


{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 0.8,
 'gamma': 0.1,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 5,
 'min_child_weight': 2,
 'missing': None,
 'n_estimators': 200,
 'nthread': -1,
 'objective': 'binary:logistic',
 'reg_alpha': 0.0,
 'reg_lambda': 1.0,
 'scale_pos_weight': 0.6229508196721312,
 'seed': 0,
 'silent': True,
 'subsample': 0.8}

In [10]:
param_grid = {
    'max_depth': range(3, 10, 2),
    'min_child_weight': range(1, 6, 2)
}

clf = XGBClassifier(**best_params) # в качестве отправной точки возьмём модель с наилучшими параметрами предыдущего шага

gs = GridSearchCV(clf, param_grid, scoring='accuracy', cv=cv, verbose=5)

gs.fit(X_train, y_train)
best_params = gs.best_estimator_.get_params()
print('Best score (AUC): ', gs.best_score_)
print('Best params: ')
best_params

Fitting 4 folds for each of 12 candidates, totalling 48 fits
[CV] max_depth=3, min_child_weight=1 .................................
[CV] .. max_depth=3, min_child_weight=1, score=0.856502, total=   0.1s
[CV] max_depth=3, min_child_weight=1 .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s


[CV] .. max_depth=3, min_child_weight=1, score=0.834081, total=   0.0s
[CV] max_depth=3, min_child_weight=1 .................................
[CV] .. max_depth=3, min_child_weight=1, score=0.816143, total=   0.0s
[CV] max_depth=3, min_child_weight=1 .................................
[CV] .. max_depth=3, min_child_weight=1, score=0.801802, total=   0.0s
[CV] max_depth=3, min_child_weight=3 .................................
[CV] .. max_depth=3, min_child_weight=3, score=0.856502, total=   0.0s
[CV] max_depth=3, min_child_weight=3 .................................
[CV] .. max_depth=3, min_child_weight=3, score=0.829596, total=   0.0s
[CV] max_depth=3, min_child_weight=3 .................................
[CV] .. max_depth=3, min_child_weight=3, score=0.834081, total=   0.0s
[CV] max_depth=3, min_child_weight=3 .................................
[CV] .. max_depth=3, min_child_weight=3, score=0.810811, total=   0.0s
[CV] max_depth=3, min_child_weight=5 .................................
[CV] .

[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:    3.7s finished


{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 0.8,
 'gamma': 0.1,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 9,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 200,
 'nthread': -1,
 'objective': 'binary:logistic',
 'reg_alpha': 0.0,
 'reg_lambda': 1.0,
 'scale_pos_weight': 0.6229508196721312,
 'seed': 0,
 'silent': True,
 'subsample': 0.8}

In [11]:
param_grid = {
    'gamma': [0.1*i for i in range(6)]
}

clf = XGBClassifier(**best_params)

gs = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=cv, verbose=5)

gs.fit(X_train, y_train)
best_params = gs.best_estimator_.get_params()
print('Best score (AUC): ', gs.best_score_)
print('Best params: ')
best_params

Fitting 4 folds for each of 6 candidates, totalling 24 fits
[CV] gamma=0.0 .......................................................
[CV] ........................ gamma=0.0, score=0.884250, total=   0.2s
[CV] gamma=0.0 .......................................................
[CV] ........................ gamma=0.0, score=0.873924, total=   0.0s
[CV] gamma=0.0 .......................................................
[CV] ........................ gamma=0.0, score=0.845367, total=   0.0s
[CV] gamma=0.0 .......................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s remaining:    0.0s


[CV] ........................ gamma=0.0, score=0.852613, total=   0.0s
[CV] gamma=0.1 .......................................................
[CV] ........................ gamma=0.1, score=0.880910, total=   0.0s
[CV] gamma=0.1 .......................................................
[CV] ........................ gamma=0.1, score=0.875818, total=   0.0s
[CV] gamma=0.1 .......................................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.4s remaining:    0.0s


[CV] ........................ gamma=0.1, score=0.845452, total=   0.0s
[CV] gamma=0.1 .......................................................
[CV] ........................ gamma=0.1, score=0.849929, total=   0.0s
[CV] gamma=0.2 .......................................................
[CV] ........................ gamma=0.2, score=0.880824, total=   0.0s
[CV] gamma=0.2 .......................................................
[CV] ........................ gamma=0.2, score=0.879346, total=   0.0s
[CV] gamma=0.2 .......................................................
[CV] ........................ gamma=0.2, score=0.841941, total=   0.0s
[CV] gamma=0.2 .......................................................
[CV] ........................ gamma=0.2, score=0.849258, total=   0.0s
[CV] gamma=0.30000000000000004 .......................................
[CV] ........ gamma=0.30000000000000004, score=0.878511, total=   0.0s
[CV] gamma=0.30000000000000004 .......................................
[CV] .

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:    2.1s finished


Best score (AUC):  0.867692369808
Best params: 


{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 0.8,
 'gamma': 0.4,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 9,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 200,
 'nthread': -1,
 'objective': 'binary:logistic',
 'reg_alpha': 0.0,
 'reg_lambda': 1.0,
 'scale_pos_weight': 0.6229508196721312,
 'seed': 0,
 'silent': True,
 'subsample': 0.8}

In [12]:
param_grid = {
    'subsample': [0.5 + 0.1*i for i in range(6)],
    'colsample_bytree': [0.5 + 0.1*i for i in range(6)]
}

clf = XGBClassifier(**best_params)

gs = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=cv, verbose=5)

gs.fit(X_train, y_train)
best_params = gs.best_estimator_.get_params()
print('Best score (AUC): ', gs.best_score_)
print('Best params: ')
best_params

Fitting 4 folds for each of 36 candidates, totalling 144 fits
[CV] colsample_bytree=0.5, subsample=0.5 .............................
[CV]  colsample_bytree=0.5, subsample=0.5, score=0.835130, total=   0.1s
[CV] colsample_bytree=0.5, subsample=0.5 .............................
[CV]  colsample_bytree=0.5, subsample=0.5, score=0.835020, total=   0.0s
[CV] colsample_bytree=0.5, subsample=0.5 .............................
[CV]  colsample_bytree=0.5, subsample=0.5, score=0.891977, total=   0.0s
[CV] colsample_bytree=0.5, subsample=0.5 .............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s remaining:    0.0s


[CV]  colsample_bytree=0.5, subsample=0.5, score=0.941073, total=   0.0s
[CV] colsample_bytree=0.5, subsample=0.6 .............................
[CV]  colsample_bytree=0.5, subsample=0.6, score=0.844296, total=   0.0s
[CV] colsample_bytree=0.5, subsample=0.6 .............................
[CV]  colsample_bytree=0.5, subsample=0.6, score=0.835106, total=   0.0s
[CV] colsample_bytree=0.5, subsample=0.6 .............................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s


[CV]  colsample_bytree=0.5, subsample=0.6, score=0.893962, total=   0.0s
[CV] colsample_bytree=0.5, subsample=0.6 .............................
[CV]  colsample_bytree=0.5, subsample=0.6, score=0.939322, total=   0.0s
[CV] colsample_bytree=0.5, subsample=0.7 .............................
[CV]  colsample_bytree=0.5, subsample=0.7, score=0.839119, total=   0.0s
[CV] colsample_bytree=0.5, subsample=0.7 .............................
[CV]  colsample_bytree=0.5, subsample=0.7, score=0.835539, total=   0.0s
[CV] colsample_bytree=0.5, subsample=0.7 .............................
[CV]  colsample_bytree=0.5, subsample=0.7, score=0.890612, total=   0.2s
[CV] colsample_bytree=0.5, subsample=0.7 .............................
[CV]  colsample_bytree=0.5, subsample=0.7, score=0.938096, total=   0.0s
[CV] colsample_bytree=0.5, subsample=0.8 .............................
[CV]  colsample_bytree=0.5, subsample=0.8, score=0.831056, total=   0.0s
[CV] colsample_bytree=0.5, subsample=0.8 ......................

[CV]  colsample_bytree=0.7, subsample=0.8, score=0.934857, total=   0.1s
[CV] colsample_bytree=0.7, subsample=0.9 .............................
[CV]  colsample_bytree=0.7, subsample=0.9, score=0.841411, total=   0.0s
[CV] colsample_bytree=0.7, subsample=0.9 .............................
[CV]  colsample_bytree=0.7, subsample=0.9, score=0.843539, total=   0.0s
[CV] colsample_bytree=0.7, subsample=0.9 .............................
[CV]  colsample_bytree=0.7, subsample=0.9, score=0.887717, total=   0.0s
[CV] colsample_bytree=0.7, subsample=0.9 .............................
[CV]  colsample_bytree=0.7, subsample=0.9, score=0.934331, total=   0.0s
[CV] colsample_bytree=0.7, subsample=1.0 .............................
[CV]  colsample_bytree=0.7, subsample=1.0, score=0.838185, total=   0.0s
[CV] colsample_bytree=0.7, subsample=1.0 .............................
[CV]  colsample_bytree=0.7, subsample=1.0, score=0.836101, total=   0.0s
[CV] colsample_bytree=0.7, subsample=1.0 ......................

[CV]  colsample_bytree=1.0, subsample=0.5, score=0.843872, total=   0.0s
[CV] colsample_bytree=1.0, subsample=0.5 .............................
[CV]  colsample_bytree=1.0, subsample=0.5, score=0.836015, total=   0.0s
[CV] colsample_bytree=1.0, subsample=0.5 .............................
[CV]  colsample_bytree=1.0, subsample=0.5, score=0.887345, total=   0.0s
[CV] colsample_bytree=1.0, subsample=0.5 .............................
[CV]  colsample_bytree=1.0, subsample=0.5, score=0.937221, total=   0.0s
[CV] colsample_bytree=1.0, subsample=0.6 .............................
[CV]  colsample_bytree=1.0, subsample=0.6, score=0.833772, total=   0.0s
[CV] colsample_bytree=1.0, subsample=0.6 .............................
[CV]  colsample_bytree=1.0, subsample=0.6, score=0.837571, total=   0.0s
[CV] colsample_bytree=1.0, subsample=0.6 .............................
[CV]  colsample_bytree=1.0, subsample=0.6, score=0.891191, total=   0.0s
[CV] colsample_bytree=1.0, subsample=0.6 ......................

[Parallel(n_jobs=1)]: Done 144 out of 144 | elapsed:   13.5s finished


{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 0.7,
 'gamma': 0.4,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 9,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 200,
 'nthread': -1,
 'objective': 'binary:logistic',
 'reg_alpha': 0.0,
 'reg_lambda': 1.0,
 'scale_pos_weight': 0.6229508196721312,
 'seed': 0,
 'silent': True,
 'subsample': 0.7}

In [13]:
param_grid = {
    'reg_alpha': [1e-5, 1e-2, 0.1, 1, 100],
    'reg_lambda': [1e-5, 1e-2, 0.1, 1, 100]
}

clf = XGBClassifier(**best_params)

gs = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=cv, verbose=5)

gs.fit(X_train, y_train)
best_params = gs.best_estimator_.get_params()
print('Best score (AUC): ', gs.best_score_)
print('Best params: ')
best_params

Fitting 4 folds for each of 25 candidates, totalling 100 fits
[CV] reg_alpha=1e-05, reg_lambda=1e-05 ...............................
[CV]  reg_alpha=1e-05, reg_lambda=1e-05, score=0.901936, total=   0.2s
[CV] reg_alpha=1e-05, reg_lambda=1e-05 ...............................
[CV]  reg_alpha=1e-05, reg_lambda=1e-05, score=0.864662, total=   0.0s
[CV] reg_alpha=1e-05, reg_lambda=1e-05 ...............................
[CV]  reg_alpha=1e-05, reg_lambda=1e-05, score=0.866349, total=   0.0s
[CV] reg_alpha=1e-05, reg_lambda=1e-05 ...............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.4s remaining:    0.0s


[CV]  reg_alpha=1e-05, reg_lambda=1e-05, score=0.854832, total=   0.0s
[CV] reg_alpha=1e-05, reg_lambda=0.01 ................................
[CV] . reg_alpha=1e-05, reg_lambda=0.01, score=0.900842, total=   0.0s
[CV] reg_alpha=1e-05, reg_lambda=0.01 ................................
[CV] . reg_alpha=1e-05, reg_lambda=0.01, score=0.865414, total=   0.0s
[CV] reg_alpha=1e-05, reg_lambda=0.01 ................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.4s remaining:    0.0s


[CV] . reg_alpha=1e-05, reg_lambda=0.01, score=0.861190, total=   0.0s
[CV] reg_alpha=1e-05, reg_lambda=0.01 ................................
[CV] . reg_alpha=1e-05, reg_lambda=0.01, score=0.857982, total=   0.0s
[CV] reg_alpha=1e-05, reg_lambda=0.1 .................................
[CV] .. reg_alpha=1e-05, reg_lambda=0.1, score=0.903956, total=   0.1s
[CV] reg_alpha=1e-05, reg_lambda=0.1 .................................
[CV] .. reg_alpha=1e-05, reg_lambda=0.1, score=0.860986, total=   0.0s
[CV] reg_alpha=1e-05, reg_lambda=0.1 .................................
[CV] .. reg_alpha=1e-05, reg_lambda=0.1, score=0.864659, total=   0.0s
[CV] reg_alpha=1e-05, reg_lambda=0.1 .................................
[CV] .. reg_alpha=1e-05, reg_lambda=0.1, score=0.853895, total=   0.2s
[CV] reg_alpha=1e-05, reg_lambda=1 ...................................
[CV] .... reg_alpha=1e-05, reg_lambda=1, score=0.901599, total=   0.0s
[CV] reg_alpha=1e-05, reg_lambda=1 ...................................
[CV] .

[CV] ..... reg_alpha=1, reg_lambda=0.01, score=0.907744, total=   0.0s
[CV] reg_alpha=1, reg_lambda=0.01 ....................................
[CV] ..... reg_alpha=1, reg_lambda=0.01, score=0.864244, total=   0.0s
[CV] reg_alpha=1, reg_lambda=0.01 ....................................
[CV] ..... reg_alpha=1, reg_lambda=0.01, score=0.870797, total=   0.0s
[CV] reg_alpha=1, reg_lambda=0.01 ....................................
[CV] ..... reg_alpha=1, reg_lambda=0.01, score=0.869860, total=   0.0s
[CV] reg_alpha=1, reg_lambda=0.1 .....................................
[CV] ...... reg_alpha=1, reg_lambda=0.1, score=0.906145, total=   0.3s
[CV] reg_alpha=1, reg_lambda=0.1 .....................................
[CV] ...... reg_alpha=1, reg_lambda=0.1, score=0.864077, total=   0.0s
[CV] reg_alpha=1, reg_lambda=0.1 .....................................
[CV] ...... reg_alpha=1, reg_lambda=0.1, score=0.871864, total=   0.0s
[CV] reg_alpha=1, reg_lambda=0.1 .....................................
[CV] .

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    8.4s finished


{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 0.7,
 'gamma': 0.4,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 9,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 200,
 'nthread': -1,
 'objective': 'binary:logistic',
 'reg_alpha': 1,
 'reg_lambda': 1e-05,
 'scale_pos_weight': 0.6229508196721312,
 'seed': 0,
 'silent': True,
 'subsample': 0.7}

In [14]:
import numpy as np
from sklearn.metrics import roc_auc_score

clf = XGBClassifier(**best_params)
best_n_estimators = clf.get_params()['n_estimators'] # возьмём наилучшие значения n_estimators с предыдущего шага
best_learning_rate = best_params['learning_rate'] # аналогичная запись
invariant_composition = best_n_estimators * best_learning_rate
n_estimators_range = [10, 30, 100, 200, 400, 600, 800, 1000]

best_score = gs.best_score_ # возьмём наилучшее качество с предыдущего шага

for n_estimators in n_estimators_range:
    learning_rate = invariant_composition / n_estimators
    clf.set_params(n_estimators=n_estimators, learning_rate=learning_rate)
    aucs = []
    for train_idx, test_idx in cv.split(X_train):
        X_train_fold, X_test_fold = X_train.iloc[train_idx], X_train.iloc[test_idx]
        y_train_fold, y_test_fold = y_train.iloc[train_idx], y_train.iloc[test_idx]
        clf.fit(X_train_fold, y_train_fold)
        preds = clf.predict_proba(X_test_fold)
        auc = roc_auc_score(y_test_fold, preds[:, 1])
        aucs.append(auc)
    auc = np.mean(aucs)
    if auc > best_score:
        best_n_estimators = n_estimators
        best_learning_rate = learning_rate
        best_score = auc
        
best_params['n_estimators'] = best_n_estimators
best_params['learning_rate'] = best_learning_rate

print('Best score (AUC): ', best_score)
print('Best params: ')
best_params

Best score (AUC):  0.879175753812
Best params: 


{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 0.7,
 'gamma': 0.4,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 9,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 200,
 'nthread': -1,
 'objective': 'binary:logistic',
 'reg_alpha': 1,
 'reg_lambda': 1e-05,
 'scale_pos_weight': 0.6229508196721312,
 'seed': 0,
 'silent': True,
 'subsample': 0.7}

In [15]:
for feature_name, feature_importance in zip(X_train.columns, clf.feature_importances_):
    print('Feature: "%s"\tFeature importance: %.4f' % (feature_name, feature_importance))

Feature: "Pclass"	Feature importance: 0.0628
Feature: "Age"	Feature importance: 0.3529
Feature: "SibSp"	Feature importance: 0.0587
Feature: "Parch"	Feature importance: 0.0382
Feature: "Fare"	Feature importance: 0.3649
Feature: "Sex_female"	Feature importance: 0.0424
Feature: "Sex_male"	Feature importance: 0.0148
Feature: "Embarked_C"	Feature importance: 0.0304
Feature: "Embarked_Q"	Feature importance: 0.0092
Feature: "Embarked_S"	Feature importance: 0.0256


In [16]:
clf=XGBClassifier(**best_params)

In [17]:
model=clf.fit(X_train, y_train)

In [18]:
pred=model.predict(X_test)

In [21]:
with open('submission.txt', 'w') as out:
    out.write('PassengerId,Survived\n')
    for passenger, y in zip(test_['PassengerId'], pred):
        out.write('%s,%s\n' % (passenger, y))